In [1]:
import numpy as np
import pandas as pd
import cv2

In [2]:
df = pd.read_csv('./classifications.csv')

In [53]:
df[df['image_filename'] == '0a2a5a681410054941cc56f51eb8fbda.png']

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y
5635,164,0a2a5a681410054941cc56f51eb8fbda.png,10.6084/m9.figshare.12230102,5636,Negative for intraepithelial lesion,1012,875
5636,164,0a2a5a681410054941cc56f51eb8fbda.png,10.6084/m9.figshare.12230102,5637,Negative for intraepithelial lesion,910,900
5637,164,0a2a5a681410054941cc56f51eb8fbda.png,10.6084/m9.figshare.12230102,5638,Negative for intraepithelial lesion,1177,416
5638,164,0a2a5a681410054941cc56f51eb8fbda.png,10.6084/m9.figshare.12230102,5639,Negative for intraepithelial lesion,1254,366
5639,164,0a2a5a681410054941cc56f51eb8fbda.png,10.6084/m9.figshare.12230102,5640,LSIL,983,450


In [51]:
def crop_image_around_point(image, x, y, crop_size):
    # Calculate the top-left corner of the crop
    x_start = max(0, x - int(crop_size))
    y_start = max(0, y - int(crop_size))

    # Calculate the bottom-right corner of the crop
    x_end = min(image.shape[1], x)# + int(crop_size / 2))
    y_end = min(image.shape[0], y)# + int(crop_size / 2))

    # Crop the image
    cropped_image = image[y_start:y_end, x_start:x_end]

    return cropped_image

In [57]:
# Example usage:
# Load your image (replace 'your_image_path.jpg' with your actual image file path)
original_image = cv2.imread('./dataset/a.png')

if original_image == None:
    print("erro")
else:
    # Replace these values with the actual x, y coordinates and the desired crop size
    center_x = 1012
    center_y = 875
    crop_size = 100

    # Crop the image around the specified x, y coordinates
    cropped_image = crop_image_around_point(original_image, center_x,  center_y, crop_size)

    # Display the original and cropped images
    cv2.imwrite('./tests/ouput.png', cropped_image)

erro


[ WARN:0@2062.898] global loadsave.cpp:248 findDecoder imread_('./dataset/a.png'): can't open/read file: check file path/integrity
